In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Spark DataFrame

In [3]:
# Read `people.json`
from pyspark.sql import SparkSession

spark_df = SparkSession.builder.appName("ReadDataFrame").getOrCreate()

person_df = spark_df.read.json("/content/drive/My Drive/people.json")

In [4]:
person_df.show()

+----+-------+
| age|   name|
+----+-------+
|NULL|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [8]:
# Show head
person_df.head()

Row(age=None, name='Michael')

In [6]:
# Show schema
person_df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [9]:
# Show column
person_df.columns

['age', 'name']

In [10]:
# Describe
person_df.describe()

DataFrame[summary: string, age: string, name: string]

In [11]:
# Describe and show
person_df.describe().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   NULL|
| stddev|7.7781745930520225|   NULL|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



In [12]:
from pyspark.sql.types import (StructField, StringType,
                               IntegerType, StructType)

In [13]:
#age: name of variable, type of variable, True/False: if it can contain null
data_schema = [StructField('age', IntegerType(), True),
               StructField('name', StringType(), True)]

In [14]:
data_schema

[StructField('age', IntegerType(), True),
 StructField('name', StringType(), True)]

In [15]:
final_struc = StructType(fields=data_schema)

In [16]:
final_struc

StructType([StructField('age', IntegerType(), True), StructField('name', StringType(), True)])

In [17]:
df = spark_df.read.json('/content/drive/My Drive/people.json', schema=final_struc)

In [18]:
df.printSchema()
#Now it has corrected the age as integer instead of long

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



In [19]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|NULL|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [20]:
df['age']

Column<'age'>

In [31]:
type(df['age'])

pyspark.sql.column.Column

In [22]:
#If you want to select a dataframe column
df.select('age')

DataFrame[age: int]

In [23]:
#If you want to view the results of dataframe column
df.select('age').show()

+----+
| age|
+----+
|NULL|
|  30|
|  19|
+----+



In [24]:
#If you want to view selected rows of dataframe
df.head(2)

[Row(age=None, name='Michael'), Row(age=30, name='Andy')]

In [25]:
df.head(2)[0]

Row(age=None, name='Michael')

In [26]:
df.head(2)[0]['name']

'Michael'

In [27]:
type(df.head(2)[0])

pyspark.sql.types.Row

In [32]:
type(df.head(2)[0]['name'])

str

In [ ]:
##The reason there are so many datatypes and objects in Spark because of its ability
#to read from a distributed source and map that out to compute


In [33]:
#selecting multiple columns
df.select(['age', 'name'])

DataFrame[age: int, name: string]

In [34]:
# Show the dataframe
df.show()

+----+-------+
| age|   name|
+----+-------+
|NULL|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [35]:
#Creating new columns `newAge`
df.withColumn('newAge', df['age']).show()

+----+-------+------+
| age|   name|newAge|
+----+-------+------+
|NULL|Michael|  NULL|
|  30|   Andy|    30|
|  19| Justin|    19|
+----+-------+------+



In [36]:
#Creating new columns `doubleAge`
df.withColumn('doubleAge', df['age']*2).show()

+----+-------+---------+
| age|   name|doubleAge|
+----+-------+---------+
|NULL|Michael|     NULL|
|  30|   Andy|       60|
|  19| Justin|       38|
+----+-------+---------+



In [ ]:
#The above operations are not inplace so everytime you make
#any changes you have to save them to a new variable

In [37]:
#Renaming a column 'age' by 'my_new_age'
df.withColumnRenamed('age', 'my_new_age').show()

+----------+-------+
|my_new_age|   name|
+----------+-------+
|      NULL|Michael|
|        30|   Andy|
|        19| Justin|
+----------+-------+



## Working with Spark SQL queries

In [38]:
#In order to work with SQL queries we need to register spark
#As a temporary SQL view
#We do CreateOrReplace incase we have already existing view it replaces

df.createOrReplaceTempView('people')

In [40]:
#Now you can pass direct SQL queries
results = spark_df.sql("SELECT * FROM people")

In [ ]:
results.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [41]:
# Select people with age = 30
result = spark_df.sql("SELECT * FROM people WHERE age=30")

In [42]:
# Show
result.show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



## Working with JSON, CSV and other formats

In [43]:
from pyspark.sql import SparkSession

In [44]:
spark = SparkSession.builder.appName('ops').getOrCreate()

In [46]:
#infer schema option is available with CSV
appl_stock_csv_df = spark_df.read.csv("/content/drive/My Drive/appl_stock.csv", header=True, inferSchema=True) # Assuming your data is in people.csv
appl_stock_csv_df.show()

+----------+------------------+------------------+------------------+------------------+---------+------------------+
|      Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+----------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|         27.464034|
|2010-01-11|212.79999700000002|        213.000002|      

In [47]:
appl_stock_csv_df.printSchema()


root
 |-- Date: date (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [48]:
# Head 3
appl_stock_csv_df.head(3)

[Row(Date=datetime.date(2010, 1, 4), Open=213.429998, High=214.499996, Low=212.38000099999996, Close=214.009998, Volume=123432400, Adj Close=27.727039),
 Row(Date=datetime.date(2010, 1, 5), Open=214.599998, High=215.589994, Low=213.249994, Close=214.379993, Volume=150476200, Adj Close=27.774976000000002),
 Row(Date=datetime.date(2010, 1, 6), Open=214.379993, High=215.23, Low=210.750004, Close=210.969995, Volume=138040000, Adj Close=27.333178000000004)]

In [49]:
%%time
appl_stock_csv_df.head(1)[0]

CPU times: user 5.75 ms, sys: 2.02 ms, total: 7.77 ms
Wall time: 119 ms


Row(Date=datetime.date(2010, 1, 4), Open=213.429998, High=214.499996, Low=212.38000099999996, Close=214.009998, Volume=123432400, Adj Close=27.727039)

In [50]:
appl_stock_csv_df.show()

+----------+------------------+------------------+------------------+------------------+---------+------------------+
|      Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+----------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|         27.464034|
|2010-01-11|212.79999700000002|        213.000002|      

In [52]:
#You can pass in conditions 'Close < 500'
appl_stock_csv_df.filter("Close < 500").select("Open").show()

+------------------+
|              Open|
+------------------+
|        213.429998|
|        214.599998|
|        214.379993|
|            211.75|
|        210.299994|
|212.79999700000002|
|209.18999499999998|
|        207.870005|
|210.11000299999998|
|210.92999500000002|
|        208.330002|
|        214.910006|
|        212.079994|
|206.78000600000001|
|202.51000200000001|
|205.95000100000001|
|        206.849995|
|        204.930004|
|        201.079996|
|192.36999699999998|
+------------------+
only showing top 20 rows



In [53]:
#Opening price of every stock where Close price is less than 500
appl_stock_csv_df.filter("Close < 500").select(['Open']).show()

+------------------+
|              Open|
+------------------+
|        213.429998|
|        214.599998|
|        214.379993|
|            211.75|
|        210.299994|
|212.79999700000002|
|209.18999499999998|
|        207.870005|
|210.11000299999998|
|210.92999500000002|
|        208.330002|
|        214.910006|
|        212.079994|
|206.78000600000001|
|202.51000200000001|
|205.95000100000001|
|        206.849995|
|        204.930004|
|        201.079996|
|192.36999699999998|
+------------------+
only showing top 20 rows



In [54]:
# Select and show 'Open' and 'Close' with 'Close < 500'
appl_stock_csv_df.filter("Close < 500").select(['Open', 'Close']).show()

+------------------+------------------+
|              Open|             Close|
+------------------+------------------+
|        213.429998|        214.009998|
|        214.599998|        214.379993|
|        214.379993|        210.969995|
|            211.75|            210.58|
|        210.299994|211.98000499999998|
|212.79999700000002|210.11000299999998|
|209.18999499999998|        207.720001|
|        207.870005|        210.650002|
|210.11000299999998|            209.43|
|210.92999500000002|            205.93|
|        208.330002|        215.039995|
|        214.910006|            211.73|
|        212.079994|        208.069996|
|206.78000600000001|            197.75|
|202.51000200000001|        203.070002|
|205.95000100000001|        205.940001|
|        206.849995|        207.880005|
|        204.930004|        199.289995|
|        201.079996|        192.060003|
|192.36999699999998|        194.729998|
+------------------+------------------+
only showing top 20 rows



In [55]:
# Select and show 'Open' and 'Close' with 'Open < 500'
appl_stock_csv_df.filter("Open < 500").select(['Open', 'Close']).show()

+------------------+------------------+
|              Open|             Close|
+------------------+------------------+
|        213.429998|        214.009998|
|        214.599998|        214.379993|
|        214.379993|        210.969995|
|            211.75|            210.58|
|        210.299994|211.98000499999998|
|212.79999700000002|210.11000299999998|
|209.18999499999998|        207.720001|
|        207.870005|        210.650002|
|210.11000299999998|            209.43|
|210.92999500000002|            205.93|
|        208.330002|        215.039995|
|        214.910006|            211.73|
|        212.079994|        208.069996|
|206.78000600000001|            197.75|
|202.51000200000001|        203.070002|
|205.95000100000001|        205.940001|
|        206.849995|        207.880005|
|        204.930004|        199.289995|
|        201.079996|        192.060003|
|192.36999699999998|        194.729998|
+------------------+------------------+
only showing top 20 rows



In [56]:
# Select and show 'Volume' with 'Close < 500'
appl_stock_csv_df.filter("Close < 500").select(['Volume']).show()

+---------+
|   Volume|
+---------+
|123432400|
|150476200|
|138040000|
|119282800|
|111902700|
|115557400|
|148614900|
|151473000|
|108223500|
|148516900|
|182501900|
|153038200|
|152038600|
|220441900|
|266424900|
|466777500|
|430642100|
|293375600|
|311488100|
|187469100|
+---------+
only showing top 20 rows



In [57]:
# Multiple conditions 'Close' < 200  and 'Open' > 200
appl_stock_csv_df.filter((appl_stock_csv_df['Close'] < 200) & (appl_stock_csv_df['Open'] > 200)).show()

+----------+------------------+----------+----------+----------+---------+------------------+
|      Date|              Open|      High|       Low|     Close|   Volume|         Adj Close|
+----------+------------------+----------+----------+----------+---------+------------------+
|2010-01-22|206.78000600000001|207.499996|    197.16|    197.75|220441900|         25.620401|
|2010-01-28|        204.930004|205.500004|198.699995|199.289995|293375600|25.819922000000002|
|2010-01-29|        201.079996|202.199995|190.250002|192.060003|311488100|         24.883208|
+----------+------------------+----------+----------+----------+---------+------------------+



In [58]:
# Multiple conditions 'Close' < 200  and 'Open' <= 200
appl_stock_csv_df.filter((appl_stock_csv_df['Close'] < 200) & (appl_stock_csv_df['Open'] <= 200)).show()

+----------+------------------+------------------+------------------+------------------+---------+------------------+
|      Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+----------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-02-01|192.36999699999998|             196.0|191.29999899999999|        194.729998|187469100|         25.229131|
|2010-02-02|        195.909998|        196.319994|193.37999299999998|        195.859997|174585600|25.375532999999997|
|2010-02-03|        195.169994|        200.200003|        194.420004|        199.229994|153832000|25.812148999999998|
|2010-02-04|        196.730003|        198.370001|        191.570005|        192.050003|189413000|         24.881912|
|2010-02-05|192.63000300000002|             196.0|        190.850002|        195.460001|212576700|25.323710000000002|
|2010-02-08|        195.690006|197.88000300000002|      

In [60]:
appl_stock_csv_df.filter(appl_stock_csv_df['Low'] == 197.16).show()

+----------+------------------+----------+------+------+---------+---------+
|      Date|              Open|      High|   Low| Close|   Volume|Adj Close|
+----------+------------------+----------+------+------+---------+---------+
|2010-01-22|206.78000600000001|207.499996|197.16|197.75|220441900|25.620401|
+----------+------------------+----------+------+------+---------+---------+



## GroupBy, Aggregate and Sorting Functions

In [61]:
spark = SparkSession.builder.appName('aggs').getOrCreate()

In [64]:
sales_info_csv_df = spark.read.csv('/content/drive/My Drive/sales_info.csv', inferSchema=True, header=True)

In [65]:
sales_info_csv_df.count()

12

In [66]:
sales_info_csv_df.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: double (nullable = true)



In [67]:
sales_info_csv_df.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



In [68]:
# groupBy 'Company'and show the counting
sales_info_csv_df.groupBy('Company').count().show()

+-------+-----+
|Company|count|
+-------+-----+
|   APPL|    4|
|   GOOG|    3|
|     FB|    2|
|   MSFT|    3|
+-------+-----+



In [69]:
# groupBy 'Company'and show the mean
sales_info_csv_df.groupBy('Company').mean().show()

+-------+-----------------+
|Company|       avg(Sales)|
+-------+-----------------+
|   APPL|            370.0|
|   GOOG|            220.0|
|     FB|            610.0|
|   MSFT|322.3333333333333|
+-------+-----------------+



In [70]:
# groupBy 'Company'and show the max
sales_info_csv_df.groupBy('Company').max().show()

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   APPL|     750.0|
|   GOOG|     340.0|
|     FB|     870.0|
|   MSFT|     600.0|
+-------+----------+



In [71]:
# groupBy 'Company'and show the min
sales_info_csv_df.groupBy('Company').min().show()

+-------+----------+
|Company|min(Sales)|
+-------+----------+
|   APPL|     130.0|
|   GOOG|     120.0|
|     FB|     350.0|
|   MSFT|     124.0|
+-------+----------+



In [72]:
## May be you dont need group by, you need something
#like average sales per company

sales_info_csv_df.agg({'Sales': 'sum'}).show()

+----------+
|sum(Sales)|
+----------+
|    4327.0|
+----------+



In [73]:
from pyspark.sql.functions import countDistinct, avg, stddev

In [81]:
#Counts distinct sales value
sales_info_csv_df.agg(countDistinct("Sales")).show()

+---------------------+
|count(DISTINCT Sales)|
+---------------------+
|                   11|
+---------------------+



In [82]:
sales_info_csv_df.select(avg('Sales')).show()

+-----------------+
|       avg(Sales)|
+-----------------+
|360.5833333333333|
+-----------------+



In [83]:
#Alias - Give the column name appropriate name
sales_info_csv_df.select(avg('Sales').alias('Average Sales')).show()

+-----------------+
|    Average Sales|
+-----------------+
|360.5833333333333|
+-----------------+



In [84]:
sales_info_csv_df.select(stddev('Sales')).show()

+------------------+
|     stddev(Sales)|
+------------------+
|250.08742410799007|
+------------------+



In [85]:
#Formatting the number
from pyspark.sql.functions import format_number

In [86]:
sales_std = df.select(stddev('Sales').alias('std'))

In [87]:
sales_std.show()

+------------------+
|               std|
+------------------+
|250.08742410799007|
+------------------+



In [88]:
#number of decimal places I want to show
sales_std.select(format_number('std', 2)).show()

+---------------------+
|format_number(std, 2)|
+---------------------+
|               250.09|
+---------------------+



In [92]:
#Ordering the columns, default is ascending order
sales_info_csv_df.orderBy('Sales', ascending = True).show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|Charlie|120.0|
|   MSFT|    Amy|124.0|
|   APPL|  Linda|130.0|
|   GOOG|    Sam|200.0|
|   MSFT|Vanessa|243.0|
|   APPL|   John|250.0|
|   GOOG|  Frank|340.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   MSFT|   Tina|600.0|
|   APPL|   Mike|750.0|
|     FB|   Carl|870.0|
+-------+-------+-----+



In [93]:
#Ordering in descending
sales_info_csv_df.orderBy(sales_info_csv_df['Sales'].desc()).show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|     FB|   Carl|870.0|
|   APPL|   Mike|750.0|
|   MSFT|   Tina|600.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   GOOG|  Frank|340.0|
|   APPL|   John|250.0|
|   MSFT|Vanessa|243.0|
|   GOOG|    Sam|200.0|
|   APPL|  Linda|130.0|
|   MSFT|    Amy|124.0|
|   GOOG|Charlie|120.0|
+-------+-------+-----+



## Dealing with Missing Data in Spark

In [94]:
spark = SparkSession.builder.appName('miss').getOrCreate()

In [121]:
miss_df = spark.read.csv('/content/drive/My Drive/ContainsNull.csv', header=True, inferSchema=True)

In [97]:
miss_df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| NULL|
|emp2| NULL| NULL|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [98]:
#You can drop all the missing data
miss_df.na.drop().show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [99]:
miss_df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| NULL|
|emp2| NULL| NULL|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [100]:
#When we specify thresh=2, it looks at the row if it contains more than 2 non-null values, it drops
miss_df.na.drop(thresh=2).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| NULL|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [101]:
#By default how is set to 'any', meaning drop if there are any null values
miss_df.na.drop(how='any').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [102]:
#We can change the how to 'all', if row has all null values only then drop
miss_df.na.drop(how='all').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| NULL|
|emp2| NULL| NULL|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [104]:
#The subset method will only drop if Sales colum has null values
miss_df.dropna(subset=['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [103]:
miss_df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sales: double (nullable = true)



In [105]:
#We can also fill NULL values
miss_df.na.fill('FILL VALUE').show()

+----+----------+-----+
|  Id|      Name|Sales|
+----+----------+-----+
|emp1|      John| NULL|
|emp2|FILL VALUE| NULL|
|emp3|FILL VALUE|345.0|
|emp4|     Cindy|456.0|
+----+----------+-----+



In [106]:
#Spark is smart enough to recognize string datatype and only fill
#in string datatype column, similar thing it does for numeric values
miss_df.na.fill(0).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|  0.0|
|emp2| NULL|  0.0|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [108]:
from pyspark.sql.functions import mean

In [122]:
mean_val = miss_df.select(mean(miss_df['Sales'])).collect()
mean_val

[Row(avg(Sales)=400.5)]

In [123]:
mean_val[0]

Row(avg(Sales)=400.5)

In [124]:
mean_val[0][0]

400.5

In [125]:
mean_sales = mean_val[0][0]

In [127]:
miss_df.na.fill(mean_sales, ['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| NULL|400.5|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [130]:
#Doing all this in one line
miss_df.na.fill(miss_df.select(mean(miss_df['Sales'])).collect()[0][0], ['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| NULL|400.5|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



## Dates and Timestamps

In [131]:
spark = SparkSession.builder.appName('dates').getOrCreate()

In [133]:
df = spark.read.csv('/content/drive/My Drive/appl_stock.csv', header=True, inferSchema=True)

In [134]:
df.count()

1762

In [135]:
df.head(1)
#datetime object in Date column, this looks messy here

[Row(Date=datetime.date(2010, 1, 4), Open=213.429998, High=214.499996, Low=212.38000099999996, Close=214.009998, Volume=123432400, Adj Close=27.727039)]

In [136]:
df.select(['Date', 'Open']).show()

+----------+------------------+
|      Date|              Open|
+----------+------------------+
|2010-01-04|        213.429998|
|2010-01-05|        214.599998|
|2010-01-06|        214.379993|
|2010-01-07|            211.75|
|2010-01-08|        210.299994|
|2010-01-11|212.79999700000002|
|2010-01-12|209.18999499999998|
|2010-01-13|        207.870005|
|2010-01-14|210.11000299999998|
|2010-01-15|210.92999500000002|
|2010-01-19|        208.330002|
|2010-01-20|        214.910006|
|2010-01-21|        212.079994|
|2010-01-22|206.78000600000001|
|2010-01-25|202.51000200000001|
|2010-01-26|205.95000100000001|
|2010-01-27|        206.849995|
|2010-01-28|        204.930004|
|2010-01-29|        201.079996|
|2010-02-01|192.36999699999998|
+----------+------------------+
only showing top 20 rows



In [138]:
from pyspark.sql.functions import (dayofmonth, hour,
                                   dayofyear, month,
                                   year, weekofyear,
                                  format_number, date_format)

In [139]:
#Whenever you are using any pyspark imported function,
#you can directly use them in select on your column
df.select(dayofmonth(df['Date'])).show()

+----------------+
|dayofmonth(Date)|
+----------------+
|               4|
|               5|
|               6|
|               7|
|               8|
|              11|
|              12|
|              13|
|              14|
|              15|
|              19|
|              20|
|              21|
|              22|
|              25|
|              26|
|              27|
|              28|
|              29|
|               1|
+----------------+
only showing top 20 rows



In [140]:
df.select(hour(df['Date'])).show()

+----------+
|hour(Date)|
+----------+
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
+----------+
only showing top 20 rows



In [141]:
df.select(month(df['Date'])).show()

+-----------+
|month(Date)|
+-----------+
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          2|
+-----------+
only showing top 20 rows



In [142]:
## May be you want to know average closing price per year
df.select(year(df['Date'])).show()

+----------+
|year(Date)|
+----------+
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
+----------+
only showing top 20 rows



In [143]:
# Take the year
df.withColumn("Year", year(df['Date'])).show()

+----------+------------------+------------------+------------------+------------------+---------+------------------+----+
|      Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|Year|
+----------+------------------+------------------+------------------+------------------+---------+------------------+----+
|2010-01-04|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|2010|
|2010-01-05|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|2010|
|2010-01-06|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|2010|
|2010-01-07|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|2010|
|2010-01-08|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|         27.464034|2010|
|2010-01-11|212.

In [148]:
df.show()

+----------+------------------+------------------+------------------+------------------+---------+------------------+
|      Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+----------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|         27.464034|
|2010-01-11|212.79999700000002|        213.000002|      

In [151]:
# Create new column `Year`
df.withColumn("Year", year(df['Date'])).groupBy('Year').mean().show()

+----+------------------+------------------+------------------+------------------+--------------------+------------------+---------+
|Year|         avg(Open)|         avg(High)|          avg(Low)|        avg(Close)|         avg(Volume)|    avg(Adj Close)|avg(Year)|
+----+------------------+------------------+------------------+------------------+--------------------+------------------+---------+
|2015|120.17575393253965|121.24452385714291| 118.8630954325397|120.03999980555547|  5.18378869047619E7|115.96740080555561|   2015.0|
|2013| 473.1281355634922| 477.6389272301587|468.24710264682557| 472.6348802857143|          1.016087E8| 62.61798788492063|   2013.0|
|2014| 295.1426195357143|297.56103184523823| 292.9949599801587| 295.4023416507935| 6.315273055555555E7| 87.63583323809523|   2014.0|
|2012|     576.652720788| 581.8254008040001| 569.9211606079999| 576.0497195640002|       1.319642044E8| 74.81383696800002|   2012.0|
|2016|104.50777772619044| 105.4271825436508|103.69027771825397|104.60

In [152]:
# Groupby `Year` and average the Close price
df.withColumn("Year", year(df['Date'])).groupBy('Year').mean().select('avg(Close)','Year').show()

+------------------+----+
|        avg(Close)|Year|
+------------------+----+
|120.03999980555547|2015|
| 472.6348802857143|2013|
| 295.4023416507935|2014|
| 576.0497195640002|2012|
|104.60400786904763|2016|
| 259.8424600000002|2010|
|364.00432532142867|2011|
+------------------+----+



## Thank you!